## *Amharic Hate Speech Detection using BERT <br> (multilingual Bidirectional Encoder Representations from Transformer)*



In [ ]:
#installing the transformers package
!pip install transformers

In [ ]:
#installing the datasets package
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00


In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.8 MB/s eta 0:00:00


In [ ]:
from datasets import Dataset
import datasets
from datasets import load_metric
import numpy as np
import pandas as pd
from google.colab import drive
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback, IntervalStrategy
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
Labels=pd.read_csv('/content/drive/MyDrive/Labels.txt',header=None)
Posts=pd.read_csv('/content/drive/MyDrive/Texts.txt',header=None)

In [ ]:
Labels.columns = ["labels"]
Posts.columns = ["post"]

In [ ]:
Labels = Labels.replace(['Free', 'Free ','Hate'],[0,0,1])

In [ ]:
Labels.head(10)

,labels
0,0
1,0
2,0
3,0
4,1
5,0
6,0
7,0
8,1
9,0


In [ ]:
Posts.head(1000)

,post
0,አስቀድሜ ጥያቄዬ በጨዋነት በውስጥ መስመር እንዲደርስዎ አድርጌ ፍትህን ለ...
1,እነዚህን ወሳኝ ጉዳዮችን የሚያስፈፅም አካል እንዲቋቋምና ክትትል እንዲደ...
2,የአማራ ህዝብ በአእምሮ ክንፉ ያልበረረበት ጥበብና ፍልስፍና ያልከፈተው የ...
3,ከአማራ ህዝብ የሀገሪቱ ዘርፈ ብዙ እውቀት መንጭቶ የሞላበትከሙላቱም በመል...
4,ዛሬ በየትኛውም መለኪያ ይሁን መመዘኛ ኢትዮጵያዊነት የሚንፀባረቀው በአማራ...
...,...
995,መፈንቅለመንግስት ከሽፏልእንዴ የየክልሉ ህዝብ ምን ይጠብቃል ሆ ብሎ ሄዶ ...
996,ሰው በሩን እንደፈለገ መክፈት እንደፈለገ መዝጋት ይችላል የምን አውቅልሻ...
997,ክክክክክ የሱዳን ዜጋ ናችሁ እንዴ ባለ ነጠላ ጎጃሜ ሁላ
998,የምን ማለቃቀስ ነው ምድረ ፎካሪ ሁላ


In [ ]:
#merge the datasets
Frames = [Labels, Posts]
Merged = pd.concat(Frames, axis=1)

In [ ]:
#preview of merged data
Merged

,labels,post
0,0,አስቀድሜ ጥያቄዬ በጨዋነት በውስጥ መስመር እንዲደርስዎ አድርጌ ፍትህን ለ...
1,0,እነዚህን ወሳኝ ጉዳዮችን የሚያስፈፅም አካል እንዲቋቋምና ክትትል እንዲደ...
2,0,የአማራ ህዝብ በአእምሮ ክንፉ ያልበረረበት ጥበብና ፍልስፍና ያልከፈተው የ...
3,0,ከአማራ ህዝብ የሀገሪቱ ዘርፈ ብዙ እውቀት መንጭቶ የሞላበትከሙላቱም በመል...
4,1,ዛሬ በየትኛውም መለኪያ ይሁን መመዘኛ ኢትዮጵያዊነት የሚንፀባረቀው በአማራ...
...,...,...
29995,1,በአሉ የሁሉም ኢትዮጵያዊ ስላልሆነ በኦሮምኛው ቢለፋደድ ምን አገባን
29996,0,ተባረክ አብቹ ፈር ቀዳጅ ስለሆንህ መጋረጃው መቀደድ ስለጀመረ
29997,0,እስከ አሁን አንተ ብቻ ነው በ መፅሀፍ ያልቻልከው አንተም ታሪክ እ...
29998,1,ህገወጥት ጠቅላይ ሚንስትር ፅቤት የተፈቀደ ሆኖ ህዝብን እንዴት ህግ አክ...


In [ ]:
#Divide the dataset into train, validation and test categories
train_val_df, test_dataset = train_test_split(Merged, test_size=0.20, random_state=42)
train_dataset, evaluation_dataset = train_test_split(train_val_df, test_size=0.115, random_state=42)
print('Training dataset shape: ', train_dataset.shape)
print('Validation dataset shape: ', evaluation_dataset.shape)
print('Testing dataset shape: ', test_dataset.shape)

Training dataset shape:  (21240, 2)
Validation dataset shape:  (2760, 2)
Testing dataset shape:  (6000, 2)


In [ ]:
#convert format of the dataset to HuggingFace Dataset from Pandas DataFrame
test_dataset=Dataset.from_pandas(test_dataset)

In [ ]:
#convert the format of the dataset to HuggingFace Dataset from Pandas DataFrame
train_dataset=Dataset.from_pandas(train_dataset)

In [ ]:
#convert the format of the dataset to HuggingFace Dataset from Pandas DataFrame
evaluation_dataset=Dataset.from_pandas(evaluation_dataset)

In [ ]:
#preview of the dataset after conversion
print(test_dataset)

Dataset({
    features: ['labels', 'post', '__index_level_0__'],
    num_rows: 6000
})


In [ ]:
#remove unnecessary column
test_dataset=test_dataset.remove_columns("__index_level_0__")
train_dataset=train_dataset.remove_columns("__index_level_0__")
evaluation_dataset=evaluation_dataset.remove_columns("__index_level_0__")

In [ ]:
#combine the train and test dataset into one datset
main_dataset= datasets.DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'evaluate': evaluation_dataset
})

In [ ]:
#preview of the dataset after merging
main_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'post'],
        num_rows: 21240
    })
    test: Dataset({
        features: ['labels', 'post'],
        num_rows: 6000
    })
    evaluate: Dataset({
        features: ['labels', 'post'],
        num_rows: 2760
    })
})

In [ ]:
# training and testing data size
training_data_size = 500
testing_data_size = 200
evaluation_data_size = 200

In [ ]:
#loading a tokenizer from the pretrained model
tokenizer = AutoTokenizer.from_pretrained("Davlan/bert-base-multilingual-cased-finetuned-amharic")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/798 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.55M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
#Have a tokenizer function that uses the tokenizer
def tokenize_function(data):
    return tokenizer(data["post"], padding="max_length", truncation=True)

In [ ]:
#Tokenize all the data using the mapping functionality
tokenized_datasets = main_dataset.map(tokenize_function)


Map:   0%|          | 0/21240 [00:00<?, ? examples/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2760 [00:00<?, ? examples/s]

In [ ]:
#empty cache
torch.cuda.empty_cache()

In [ ]:
#remove the posts column as it is no longer needed
tokenized_datasets = tokenized_datasets.remove_columns(["post"])

In [ ]:
#changing the format of the tokenized dataset to torch
tokenized_datasets.set_format("torch")

In [ ]:
#shuffeling and selecting the needed size of dataset for training and evaluating the model
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(training_data_size))
small_test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(testing_data_size))
small_eval_dataset = tokenized_datasets["evaluate"].shuffle(seed=42).select(range(evaluation_data_size))

In [ ]:
# preview of the shuffeled and selected evaluation dataset
small_eval_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 200
})

In [ ]:
# preview of the shuffeled and selected training dataset
small_train_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 500
})

In [ ]:
# preview of the shuffeled and selected testing dataset
small_test_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 200
})

In [ ]:
#load the dataset using DataLoader
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=4)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=4)
test_dataloader = DataLoader(small_test_dataset, batch_size=4)

In [ ]:
#Load auto mode classifier from the pretrained model
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("Davlan/bert-base-multilingual-cased-finetuned-amharic", num_labels=2)
# model = AutoModelForSequenceClassification.from_pretrained("Davlan/", num_labels=2)

pytorch_model.bin:   0%|          | 0.00/712M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Davlan/bert-base-multilingual-cased-finetuned-amharic and are newly initialized: ['bert.pooler.dense.bias', 'classifier.weight', 'classifier.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1","accuracy")

<ipython-input-40-a9d1bd3d149e>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("f1","accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
#Function that uses the loaded metrics to compute the performance of the model
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
!pip install transformers[torch]

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
from transformers import EarlyStoppingCallback, IntervalStrategy

In [ ]:
training_args = TrainingArguments(
   f"training_with_callbacks",
   evaluation_strategy = IntervalStrategy.STEPS, # "steps"
   warmup_steps=500,                # number of warmup steps for learning rate
   save_steps=2000,
   eval_steps = 2000, # Evaluation and Save happens every 2000 steps
   save_total_limit = 3, # Only last 3 models are saved. Older ones are deleted.
   learning_rate=1e-5,
   per_device_train_batch_size=4,
   per_device_eval_batch_size=4,
   num_train_epochs=15,
   weight_decay=0.01,
   push_to_hub=False,
   metric_for_best_model = 'f1',
   do_predict=True,
   load_best_model_at_end=True)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=10)],
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=1875, training_loss=0.2889645787556966, metrics={'train_runtime': 858.6114, 'train_samples_per_second': 8.735, 'train_steps_per_second': 2.184, 'total_flos': 1973332915200000.0, 'train_loss': 0.2889645787556966, 'epoch': 15.0})

In [ ]:
trainer.evaluate(small_test_dataset)

{'eval_loss': 3.5037262439727783,
 'eval_f1': 0.6285714285714286,
 'eval_runtime': 7.8987,
 'eval_samples_per_second': 25.321,
 'eval_steps_per_second': 6.33,
 'epoch': 15.0}

In [ ]:
# Define a function to make predictions on new text
model = trainer.model
def predict_text(text):
    inputs = tokenizer([text], return_tensors="pt").to(device=model.device)

    model.to(inputs['input_ids'].device)

    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()


    return predicted_class

text_to_predict = "የተባረከ"
prediction = predict_text(text_to_predict)
print("Predicted Class:", prediction)

Predicted Class: 1


In [ ]:
from joblib import dump

# Save the entire model, including architecture and parameters
dump(model, 'bert_model.joblib')

['bert_model.joblib']